In [1]:
import os
import numpy as np
import torch
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
# from torchvision.transforms import ToTensor
import torchvision.transforms as transforms
from torch.nn import Module
from torch.nn import Linear, CosineSimilarity, BatchNorm1d, BatchNorm2d, Sequential, Dropout, ReLU, LeakyReLU, Conv2d, MaxPool2d, AvgPool2d, AdaptiveAvgPool2d, CrossEntropyLoss

In [2]:
cuda = True
batchsize = 256 if cuda else 64
num_workers = 4 if cuda else 0
# from Recitation 5
# not doing horizontal flip as features can be side dependent, not doing RandomGrayScale to start
transformations = transforms.Compose([
#     transforms.Resize((68,68)),
#     transforms.RandomCrop((64,64)), # this should help with translation variants
#     transforms.ColorJitter(brightness=0.09, contrast=0.09, saturation=0.09),  
#     transforms.RandomRotation(degrees=5), #20
    transforms.ToTensor()
])
train_dataset = ImageFolder(root='classification_data/train_data/', transform=transformations)#ToTensor())
droplast=False
if len(train_dataset)%batchsize==1:
    print("Dropping last because last batch of size:", len(train_dataset)%batchsize)
    droplast=True
train_loader_args = dict(batch_size=batchsize, shuffle=True, num_workers=num_workers, pin_memory=True, drop_last=droplast) if cuda else dict(shuffle=True, batch_size=batchsize, drop_last=droplast)
train_loader = DataLoader(train_dataset, **train_loader_args)

val_dataset = ImageFolder(root='classification_data/val_data/', transform=transforms.ToTensor())
val_loader_args = dict(batch_size=batchsize, shuffle=False, num_workers=num_workers, pin_memory=True) if cuda else dict(shuffle=False, batch_size=batchsize)
val_loader = DataLoader(val_dataset, **val_loader_args)

class VerificationMyDataset(Dataset):
    def __init__(self, pair_path, test=False):
        pairs = open(pair_path, "r").readlines(-1)
        self.test = test
        self.x=[]
        if not self.test:
            self.y=[]
        self.to_tensor = transforms.ToTensor()
        self.length=len(pairs)
        for i in range(self.length):
            line = pairs[i].strip().split(' ')
            image1 = self.to_tensor(Image.open(line[0]))
            image2 = self.to_tensor(Image.open(line[1]))
            self.x.append((image1,image2))
            if not self.test:
                self.y.append(int(line[2]))
        
    def __len__(self):
        return self.length

    # keep this simple/quick because run many times
    def __getitem__(self, index):
        if not self.test:
            return self.x[index], self.y[index]
        else:
            return self.x[index]
pairs_path = "verification_pairs_val.txt"

ver_val_dataset = VerificationMyDataset(pairs_path)
ver_val_loader_args = dict(batch_size=batchsize, shuffle=False, num_workers=num_workers, pin_memory=True) if cuda else dict(shuffle=False, batch_size=batchsize)
ver_val_loader = DataLoader(ver_val_dataset, **ver_val_loader_args)

# test_data = ImageFolder(root='classification_data/test_data/', transform=ToTensor())
# test_loader = DataLoader(test_data, batch_size=batchsize, shuffle=True, num_workers=num_workers)

In [27]:
train_dataset[1000][0].shape

torch.Size([3, 64, 64])

In [3]:
# train_dataset[0][0].shape
pairs[0]

(108520, 341782, 0)

## On Deck:

In [3]:
# Model_24_ 
class Model(Module):
    def __init__(self):
        super().__init__()
        
        # HEAD
        head = [
            Conv2d(3, 64, kernel_size=3, stride=1, bias=False), # 64x64
            MaxPool2d(kernel_size=3, stride=2, padding=1) # 32x32
        ]
        self.head = Sequential(*head)
        
        # Embedder
        self.block1 = BasicBlock(64,64)
        self.block2 = BasicBlock(64,64)
        
        self.block3 = BasicBlock(64,128, stride=2) # 16x16
        self.skipdown1 = SkipDownSample(64,128, stride=2)             # MAYBE SHIFT THIS DOWN 1 SET OF BLOCKS
        self.block4 = BasicBlock(128,128)
        
        self.block5 = BasicBlock(128,256) # , stride=2) # 16x16
        self.skipdown2 = SkipDownSample(128,256) # , stride=2)
        self.block6 = BasicBlock(256,256)
        
        self.block7 = BasicBlock(256,512, stride=2) # 8x8
        self.skipdown3 = SkipDownSample(256,512, stride=2)            # MAYBE SHIFT THIS DOWN 1 SET OF BLOCKS
        self.block8 = BasicBlock(512,512)
        
        # I added
        self.block9 = BasicBlock(512, 1024) # 8x8
        self.skipdown4 = SkipDownSample(512,1024)
        
        self.block10 = BasicBlock(1024, 2048) # 8x8
        self.skipdown5 = SkipDownSample(1024,2048)

        # EMBEDDING
        self.avgpool = AvgPool2d(kernel_size=8, stride=1) # kernel size 4 -> 8
        
        # CLASSIFICATION
        self.classification = Linear(2048, 4000)
        #             Linear(1024, 4000)

        
#         ]
#         self.classification = Sequential(*classification)
    
    
    def forward(self, x):
        # HEAD
        # 64x64
        out = self.head(x)
        # 32x32
        # EMBEDDING
        out = self.block1(out) + out
        out = self.block2(out) + out
        out = self.block3(out) + self.skipdown1(out) # 16x16
        out = self.block4(out) + out
        out = self.block5(out) + self.skipdown2(out)# 8x8
        out = self.block6(out) + out
        out = self.block7(out) + self.skipdown3(out)# 4x4
        out = self.block8(out) + out
        out = self.block9(out) + self.skipdown4(out)# 2x2
        out = self.block10(out) + self.skipdown5(out)
        
        out = self.avgpool(out)
        embedding = torch.flatten(out,1)
        
        classification = self.classification(embedding)
                
        return embedding, classification

## Model:

In [4]:
# Model_18_
class BasicBlock(Module): 
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, dropout=False, drop_rate=0.3): # groups=1, dilation=1, norm_layer=None
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        layers = [
            Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=kernel_size-2, stride=stride, bias=False),  # , padding=kernel_size-1
            BatchNorm2d(out_channels),
            ReLU(),
            Conv2d(out_channels, out_channels, kernel_size=kernel_size, padding=kernel_size-2, stride=1, bias=False),  # , padding=kernel_size-1
            BatchNorm2d(out_channels),
            ReLU()
        ]
        
        if dropout:
            layers.append(Dropout(drop_rate))
            
        self.block = Sequential(*layers)

    def forward(self,x):
        out = self.block(x)
         
        return out

class SkipDownSample(Module):
    def __init__(self, in_channels, out_channels, stride=1): # groups=1, dilation=1, norm_layer=None
        super().__init__()
    
#     layers = [
#         Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False)
#     ]
    
        self.skip = Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)

    
    def forward(self, x):
        return self.skip(x)
    

In [4]:
block=BasicBlock(64,64,stride=2)
data=torch.rand((32,64,64,64))
print(block.forward(data).shape)

skip = SkipDownSample(64,64,stride=2)
data=torch.rand((32,64,64,64))
print(skip.forward(data).shape)

maxpool=MaxPool2d(kernel_size=3, stride=2, padding=1)
data=torch.rand((32,64,64,64))
print(maxpool.forward(data).shape)

torch.Size([32, 64, 32, 32])
torch.Size([32, 64, 32, 32])
torch.Size([32, 64, 32, 32])


In [5]:
# Model_26_ 
class Model(Module):
    def __init__(self):
        super().__init__()
        
        # HEAD
        head = [
            Conv2d(3, 64, kernel_size=3, stride=1, bias=False), # 64x64
            MaxPool2d(kernel_size=3, stride=2, padding=1) # 32x32
        ]
        self.head = Sequential(*head)
        
        # Embedder
        self.block1 = BasicBlock(64,64)
        self.block2 = BasicBlock(64,64)
        
        self.block3 = Sequential(BasicBlock(64,128), MaxPool2d(kernel_size=3, stride=2, padding=1))#, stride=2) # 16x16
        self.skipdown1 = SkipDownSample(64,128, stride=2)             # MAYBE SHIFT THIS DOWN 1 SET OF BLOCKS
        self.block4 = BasicBlock(128,128)
        
        self.block5 = BasicBlock(128,256) # , stride=2) # 16x16
        self.skipdown2 = SkipDownSample(128,256) # , stride=2)
        self.block6 = BasicBlock(256,256)
        
        self.block7 = BasicBlock(256,512, stride=2) # 8x8
        self.skipdown3 = SkipDownSample(256,512, stride=2)            # MAYBE SHIFT THIS DOWN 1 SET OF BLOCKS
        self.block8 = BasicBlock(512,512)
        
        # I added
        self.block9 = BasicBlock(512, 1024) # 8x8
        self.skipdown4 = SkipDownSample(512,1024)
        
        self.block10 = BasicBlock(1024,2048) # 8x8
        self.skipdown5 = SkipDownSample(1024,2048)

        # EMBEDDING
        self.avgpool = AvgPool2d(kernel_size=8, stride=1) # kernel size 4 -> 8
        
        # CLASSIFICATION
        self.classification = Linear(2048, 4000)
    
    
    def forward(self, x):
        # HEAD
        # 64x64
        out = self.head(x)
        # 32x32
        # EMBEDDING
        out = self.block1(out) + out
        out = self.block2(out) + out
        out = self.block3(out) + self.skipdown1(out) # 16x16
        out = self.block4(out) + out
        out = self.block5(out) + self.skipdown2(out)# 8x8
        out = self.block6(out) + out
        out = self.block7(out) + self.skipdown3(out)# 4x4
        out = self.block8(out) + out
        out = self.block9(out) + self.skipdown4(out)# 2x2
        out = self.block10(out) + self.skipdown5(out)
        
        out = self.avgpool(out)
        embedding = torch.flatten(out,1)
        
        classification = self.classification(embedding)
                
        return embedding, classification

model = Model()
print(model)

Model(
  (head): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (block1): BasicBlock(
    (block): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
    )
  )
  (block2): BasicBlock(
    (block): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d

## For Classification Training

In [6]:
from time import time
model=Model()
data=torch.rand((32,3,64,64))
ti=time()
out = model.forward(data)
print(out[0].shape, out[1].shape)
tf=time()
(tf-ti)*380638/(60*60)

torch.Size([32, 2048]) torch.Size([32, 4000])


184.49763504399195

In [7]:

RUN_NUMBER = 26.5  # <============================= CHANGE THIS EVERY TIME ======================<<<<<<<<<


In [8]:
from torch import optim

# model_path = 'hw2p2_models/model_26_7/model.pt'
# model = torch.load(model_path)
NUM_EPOCHS = 40
learning_rate = 1e-1
mile_stones = [3,5,6,8,12,16,20,24,28,32,36,40] # [4,5,6,8,12,16,20,24,28,32,36,40]
gamma = 0.2
optimizer = optim.SGD(model.parameters(), momentum=0.9, weight_decay=5e-5, lr=learning_rate)
# TODO: TRY RMSProp
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=mile_stones, gamma=gamma)
criterion = CrossEntropyLoss()
model.to(device)

Model(
  (head): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (block1): BasicBlock(
    (block): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
    )
  )
  (block2): BasicBlock(
    (block): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d

In [9]:
import csv
from sklearn.metrics import roc_auc_score

def validate(model, val_loader, val_dataset):
    model.eval()
    total = len(val_dataset)
    num_correct = 0
    for i, (x,y) in enumerate(val_loader):
        x = x.to(device)
        y = y.reshape(-1).to(device)

        out = model(x)[1]

        out = out.to("cpu")
        y = y.to("cpu")

        batch_predictions = np.argmax(out.data, axis=1)
        num_correct += (batch_predictions == y).sum()
    
    accuracy = num_correct.item() / total
        
    # Deallocate memory in GPU
    torch.cuda.empty_cache()
    del x
    del y
    
    model.train()
    return accuracy

def save_state(AUC, accuracy, running_max, model_number, model, train_loader_args, device, NUM_EPOCHS, learning_rate, optimizer, criterion):
    path = './hw2p2_models/model_' + str(RUN_NUMBER) + '_'+str(model_number)
    if not os.path.isdir(path):
        os.makedirs(path)
    torch.save(model, path+'/model.pt')
    # write parameter tracking file
    parameter_file = open(path+'/hyperparameters.txt', 'w')
    parameter_file.write('AUC:\n' + str(AUC))
    parameter_file.write('Accuracy:\n' + str(accuracy))
    parameter_file.write('Running Max:\n' + str(running_max[0]) + "  " + str(running_max[1]))
    parameter_file.write('\nModel:\n' + str(model))
    parameter_file.write('\ntrain_loader_args:\n' + str(train_loader_args))
    parameter_file.write('\nDevice:\n' + str(device))
    parameter_file.write('\nNUM_EPOCHS:\n' + str(NUM_EPOCHS))
    parameter_file.write('\nLearning Rate:\n' + str(learning_rate))
    parameter_file.write('\nOptimizer:\n' + str(optimizer))
    parameter_file.write('\nCriterion:\n' + str(criterion))
    parameter_file.close()
    

sub_name = './submission6.csv'
def verify(model, data_loader, sub_name, test=False):
    model.eval()
    
    cos_sim = CosineSimilarity(dim=1)
    cosine_similarity = torch.Tensor([])
    true_similarity = torch.Tensor([])
    if not test:
        for i, (x,y) in enumerate(data_loader):
            img1 = x[0].to(device)
            img2 = x[1].to(device)
            y = y.to("cpu")
            
            out1 = model(img1)[0].to("cpu")
            out2 = model(img2)[0].to("cpu")
            
            cosine_similarity = torch.cat((cosine_similarity.detach(), cos_sim(out1,out2).detach()), 0)
            true_similarity = torch.cat((true_similarity, y), 0)

            del x, y, img1, img2, out1, out2
            torch.cuda.empty_cache()
        model.train()
        try:  
            AUC = roc_auc_score(true_similarity.type(torch.DoubleTensor), cosine_similarity.type(torch.DoubleTensor).detach().numpy())
            return AUC
        except Exception as e:
            print(e)
            return -1
    else:
        for i, (x) in enumerate(data_loader):
            img1 = x[0].to(device)
            img2 = x[1].to(device)
            
            out1 = model(img1)[0].to("cpu")
            out2 = model(img2)[0].to("cpu")
            
            cosine_similarity = torch.cat((cosine_similarity.detach(), cos_sim(out1,out2).detach()), 0)
            if i%1000==0:
                print("Verification", i, end='\r')
        model.train()
        return write_submission(sub_name, cosine_similarity)

def write_submission(sub_name, cosine_similarity):
    pair_path = "verification_pairs_test.txt"
    pairs = open(pair_path, "r").readlines(-1)
    
    submission = csv.writer(open(sub_name, "w"))
    submission.writerow(['Id','Category'])

    self_length=len(pairs)
    for i in range(self_length):
        submission.writerow([pairs[i].strip(),cosine_similarity[i].item()])
        if i%1000==0:
            print("Saved {} predicitons".format((i+1)*batchsize), end='\r')

    print("Submission File COMPLETE")
    return self_length

In [10]:
validate(model, val_loader, val_dataset), verify(model, ver_val_loader, sub_name)

(0.00025, 0.5239093956700484)

In [11]:
# toggle to train so save gradients
from time import time
# val_accuracies=[]
accuracy = 0.
model_number = 0 
prev_acc = 0
running_max = ['',0.]
for epoch in range(NUM_EPOCHS):
    ti = time()
    model.train()
    for i, (x,y) in enumerate(train_loader):
#         if i%10000==0:
#             for p in model.parameters():
#                 print(p.name, p.data)
#             print(i, ':', x.shape, y.shape)
#         print("Completed", i)
        
        optimizer.zero_grad()

        x = x.to(device)
        y = y.reshape(-1).to(device) # need to turn to row

        output = model(x)[1]
        loss = criterion(output, y)

        loss.backward()
        optimizer.step()

        # progress
        if i%10==0:
            print('Epoch:', epoch, '| Iteration:', i, end='\r')

    
    # Deallocate memory in GPU
    torch.cuda.empty_cache()
    del x
    del y

    # validation
    accuracy = validate(model, val_loader, val_dataset)
    AUC = verify(model, ver_val_loader, sub_name)
#     val_accuracies.append(accuracy)

    print("Epoch", epoch, "Accuracy:", accuracy, "AUC:", AUC)
    if prev_acc == 0:
        print("\tImprovement:", accuracy-prev_acc)
    else:
        print("\tImprovement:", accuracy-prev_acc, "| Percent Improvement:", 100*(accuracy-prev_acc)/prev_acc, '%')
    # tracking running best AUC
    if running_max[1]<AUC:
        running_max[0]='Model_' + str(RUN_NUMBER) + '_' + str(epoch)
        running_max[1]=AUC
        
    if (AUC > 0.8):
        save_state(AUC, accuracy, running_max, model_number, model, train_loader_args, device, NUM_EPOCHS, learning_rate, optimizer, criterion)
    model_number+=1

    scheduler.step()
    
    tf=time()
    print("Time for epoch:", tf-ti)
    print('   Running Max:', *running_max,'\n')

    prev_acc = accuracy

	Improvement: 0.152n: 1480
Epoch 0 Accuracy: 0.152 AUC: 0.7680813549873932
Time for epoch: 2168.746775865555
   Running Max: Model_26.5_0 0.7680813549873932 

	Improvement: 0.16737500000000002 | Percent Improvement: 110.11513157894737 %
Epoch 1 Accuracy: 0.319375 AUC: 0.8650197296941574
Time for epoch: 2193.874383211136
   Running Max: Model_26.5_1 0.8650197296941574 

	Improvement: 0.17224999999999996 | Percent Improvement: 53.933463796477476 %
Epoch 2 Accuracy: 0.491625 AUC: 0.8694364456097883
Time for epoch: 2207.7084872722626
   Running Max: Model_26.5_2 0.8694364456097883 

	Improvement: 0.08337499999999998 | Percent Improvement: 16.95906432748538 %
Epoch 3 Accuracy: 0.575 AUC: 0.8555907165917824
Time for epoch: 2204.429117202759
   Running Max: Model_26.5_2 0.8694364456097883 

	Improvement: 0.16762500000000002 | Percent Improvement: 29.152173913043484 %
Epoch 4 Accuracy: 0.742625 AUC: 0.8894602141491199
Time for epoch: 2209.494797229767
   Running Max: Model_26.5_4 0.88946021414

KeyboardInterrupt: 

In [ ]:
save_state(AUC, accuracy, model_number, model, train_loader_args, device, NUM_EPOCHS, learning_rate, optimizer, criterion)
# model

## Metric Learning

In [1]:
import os
import numpy as np
from PIL import Image
from time import time
import torch
from torch import optim
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torch.nn import Module
from torch.nn import Linear, CosineSimilarity, BatchNorm1d, BatchNorm2d, Sequential, Dropout, ReLU, LeakyReLU, Conv2d, MaxPool2d, AvgPool2d, AdaptiveAvgPool2d, CrossEntropyLoss
import csv
from sklearn.metrics import roc_auc_score

def validate(model, val_loader, val_dataset):
    model.eval()
    total = len(val_dataset)
    num_correct = 0
    for i, (x,y) in enumerate(val_loader):
        x = x.to(device)
        y = y.reshape(-1).to(device)

        out = model(x)[1]

        out = out.to("cpu")
        y = y.to("cpu")

        batch_predictions = np.argmax(out.data, axis=1)
        num_correct += (batch_predictions == y).sum()
    
    accuracy = num_correct.item() / total
        
    # Deallocate memory in GPU
    torch.cuda.empty_cache()
    del x
    del y
    
    model.train()
    return accuracy

def save_state(AUC, accuracy, running_max, model_number, model, train_loader_args, device, NUM_EPOCHS, learning_rate, optimizer, criterion):
    path = './hw2p2_models/model_' + str(RUN_NUMBER) + '_'+str(model_number)
    if not os.path.isdir(path):
        os.makedirs(path)
    torch.save(model, path+'/model.pt')
    # write parameter tracking file
    parameter_file = open(path+'/hyperparameters.txt', 'w')
    parameter_file.write('AUC:\n' + str(AUC))
    parameter_file.write('Accuracy:\n' + str(accuracy))
    parameter_file.write('Running Max:\n' + str(running_max[0]) +" "+ str(running_max[1]))
    parameter_file.write('\nModel:\n' + str(model))
    parameter_file.write('\ntrain_loader_args:\n' + str(train_loader_args))
    parameter_file.write('\nDevice:\n' + str(device))
    parameter_file.write('\nNUM_EPOCHS:\n' + str(NUM_EPOCHS))
    parameter_file.write('\nLearning Rate:\n' + str(learning_rate))
    parameter_file.write('\nOptimizer:\n' + str(optimizer))
    parameter_file.write('\nCriterion:\n' + str(criterion))
    parameter_file.close()
    

sub_name = './submission1.csv'
def verify(model, data_loader, sub_name, test=False):
    model.eval()
    
    cos_sim = CosineSimilarity(dim=1)
    cosine_similarity = torch.Tensor([])
    true_similarity = torch.Tensor([])
    if not test:
        for i, (x,y) in enumerate(data_loader):
            img1 = x[0].to(device)
            img2 = x[1].to(device)
            y = y.to("cpu")
            
            out1 = model(img1)[0].to("cpu")
            out2 = model(img2)[0].to("cpu")
            
            cosine_similarity = torch.cat((cosine_similarity.detach(), cos_sim(out1,out2).detach()), 0)
            true_similarity = torch.cat((true_similarity, y), 0)

            del x, y, img1, img2, out1, out2
            torch.cuda.empty_cache()
        model.train()
        try:  
            AUC = roc_auc_score(true_similarity.type(torch.DoubleTensor), cosine_similarity.type(torch.DoubleTensor).detach().numpy())
            return AUC
        except Exception as e:
            print(e)
            return -1
    else:
        for i, (x) in enumerate(data_loader):
            img1 = x[0].to(device)
            img2 = x[1].to(device)
            
            out1 = model(img1)[0].to("cpu")
            out2 = model(img2)[0].to("cpu")
            
            cosine_similarity = torch.cat((cosine_similarity.detach(), cos_sim(out1,out2).detach()), 0)
            if i%1000==0:
                print("Verification", i, end='\r')
        model.train()
        return write_submission(sub_name, cosine_similarity)

def write_submission(sub_name, cosine_similarity):
    pair_path = "verification_pairs_test.txt"
    pairs = open(pair_path, "r").readlines(-1)
    
    submission = csv.writer(open(sub_name, "w"))
    submission.writerow(['Id','Category'])

    self_length=len(pairs)
    for i in range(self_length):
        submission.writerow([pairs[i].strip(),cosine_similarity[i].item()])
        if i%1000==0:
            print("Saved {} predicitons".format((i+1)*batchsize), end='\r')

    print("Submission File COMPLETE")
    return self_length

# ------------------------------------------------ DATASETS ------------------------------------------------ #
class TrainVerificationMyDataset(Dataset):
    def __init__(self, train_path='classification_data/train_data/', N_generated=10000, ratio_of_same=0.3):
        self.train_dataset = ImageFolder(root=train_path, transform=ToTensor())
        
        self.length_train = len(self.train_dataset)
        self.length = N_generated
        
        self.by_label = {}
        for i in range(self.length_train):
            y = self.train_dataset[i][1]
            if y in self.by_label:
                self.by_label[y].append(i)
            else:
                self.by_label[y] = [i]
            if i%1000==0:
                print("by_label {}".format(i), end='\r')
        print("by_label created         ")
        
        self.load_pairs(N_generated, ratio_of_same)
        #         del by_label, same, different, N_generated, total_same, total_different, length_train
        
    def load_pairs(self, N_generated, ratio_of_same):
        total_same = int(N_generated*(ratio_of_same))
        total_different = N_generated - total_same
        self.pairs = []
        same = 0
        different = 0
        while different < total_different:
            # randomly select an index
            index1 = torch.randint(0,self.length_train, (1,)).item()
            y1 = self.train_dataset[index1][1]
            # randomly select another index
            index2 = torch.randint(0,self.length_train, (1,)).item()
            # try not to get matches - this doesn't really matter and is unlikely
            while index2 == index1:
                index2 = torch.randint(0,self.length_train, (1,)).item()

            y2 = self.train_dataset[index2][1]

            pair_label = int(y1==y2)

            if pair_label == 1:
                same += 1
            else:
                different += 1
            self.pairs.append((index1, index2, y1, y2, pair_label))

        while same < total_same:
            # randomly select an index
            index1 = torch.randint(0,self.length_train, (1,)).item()
            y1 = self.train_dataset[index1][1]

            # randomly select same labeled item
            by_label_index = torch.randint(0,len(self.by_label[y1]), (1,)).item()
            index2 = self.by_label[y1][by_label_index]

            self.pairs.append((index1, index2, y1, y1, 1))
            same+=1
        print("Train Data Loaded: Total {} with {} matches and {} different".format(N_generated, total_same, total_different))
            
    def __len__(self):
        return self.length
    def __getitem__(self, index):
        index1, index2, y1, y2, match = self.pairs[index]
        x1 = self.train_dataset[index1][0]
        x2 = self.train_dataset[index2][0]
        return (x1, x2), (y1, y2), match

class VerificationMyDataset(Dataset):
    def __init__(self, pair_path, test=False):
        pairs = open(pair_path, "r").readlines(-1)
        self.test = test
        self.x=[]
        if not self.test:
            self.y=[]
        self.to_tensor = ToTensor()
        self.length=len(pairs)
        for i in range(self.length):
            line = pairs[i].strip().split(' ')
            image1 = self.to_tensor(Image.open(line[0]))
            image2 = self.to_tensor(Image.open(line[1]))
            self.x.append((image1,image2))
            if not self.test:
                self.y.append(int(line[2]))
    def __len__(self):
        return self.length
    def __getitem__(self, index):
        if not self.test:
            return self.x[index], self.y[index]
        else:
            return self.x[index]

In [2]:
cuda = True
batchsize = 256 if cuda else 64
num_workers = 4 if cuda else 0

val_dataset = ImageFolder(root='classification_data/val_data/', transform=ToTensor())
val_loader_args = dict(batch_size=batchsize, shuffle=False, num_workers=num_workers, pin_memory=True) if cuda else dict(shuffle=False, batch_size=batchsize)
val_loader = DataLoader(val_dataset, **val_loader_args)

ver_train_dataset = TrainVerificationMyDataset(N_generated=100000, ratio_of_same=0.5)
ver_train_loader_args = dict(batch_size=batchsize, shuffle=True, num_workers=num_workers, pin_memory=True) if cuda else dict(shuffle=True, batch_size=batchsize)
ver_train_loader = DataLoader(ver_train_dataset, **ver_train_loader_args)

pairs_path = "verification_pairs_val.txt"
ver_val_dataset = VerificationMyDataset(pairs_path)
ver_val_loader_args = dict(batch_size=batchsize, shuffle=True, num_workers=num_workers, pin_memory=True) if cuda else dict(shuffle=True, batch_size=batchsize)
ver_val_loader = DataLoader(ver_val_dataset, **ver_val_loader_args)

print("All Data Loaded")

by_label created         
Train Data Loaded: Total 100000 with 50000 matches and 50000 different
All Data Loaded


In [3]:
i = 0
print(ver_train_dataset[i][0][0].shape, ver_train_dataset[i][0][1].shape, ver_train_dataset[i][1])
for i, (x, y, match) in enumerate(ver_train_loader):
    print(i, x[0].shape, x[1].shape, y[0].shape, y[1].shape, match.shape)
    if i==2:
        break

torch.Size([3, 64, 64]) torch.Size([3, 64, 64]) (211, 3210)
0 torch.Size([256, 3, 64, 64]) torch.Size([256, 3, 64, 64]) torch.Size([256]) torch.Size([256]) torch.Size([256])
1 torch.Size([256, 3, 64, 64]) torch.Size([256, 3, 64, 64]) torch.Size([256]) torch.Size([256]) torch.Size([256])
2 torch.Size([256, 3, 64, 64]) torch.Size([256, 3, 64, 64]) torch.Size([256]) torch.Size([256]) torch.Size([256])


In [3]:
# Model_18_
class BasicBlock(Module): 
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, dropout=False, drop_rate=0.3): # groups=1, dilation=1, norm_layer=None
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        layers = [
            Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=kernel_size-2, stride=stride, bias=False),  # , padding=kernel_size-1
            BatchNorm2d(out_channels),
            ReLU(),
            Conv2d(out_channels, out_channels, kernel_size=kernel_size, padding=kernel_size-2, stride=1, bias=False),  # , padding=kernel_size-1
            BatchNorm2d(out_channels),
            ReLU()
        ]
        
        if dropout:
            layers.append(Dropout(drop_rate))
            
        self.block = Sequential(*layers)

    def forward(self,x):
        out = self.block(x)
         
        return out

class SkipDownSample(Module):
    def __init__(self, in_channels, out_channels, stride=1): # groups=1, dilation=1, norm_layer=None
        super().__init__()
    
#     layers = [
#         Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False)
#     ]
    
        self.skip = Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)

    
    def forward(self, x):
        return self.skip(x)
# Model_26_ 
class Model(Module):
    def __init__(self):
        super().__init__()
        
        # HEAD
        head = [
            Conv2d(3, 64, kernel_size=3, stride=1, bias=False), # 64x64
            MaxPool2d(kernel_size=3, stride=2, padding=1) # 32x32
        ]
        self.head = Sequential(*head)
        
        # Embedder
        self.block1 = BasicBlock(64,64)
        self.block2 = BasicBlock(64,64)
        
        self.block3 = Sequential(BasicBlock(64,128), MaxPool2d(kernel_size=3, stride=2, padding=1))#, stride=2) # 16x16
        self.skipdown1 = SkipDownSample(64,128, stride=2)             # MAYBE SHIFT THIS DOWN 1 SET OF BLOCKS
        self.block4 = BasicBlock(128,128)
        
        self.block5 = BasicBlock(128,256) # , stride=2) # 16x16
        self.skipdown2 = SkipDownSample(128,256) # , stride=2)
        self.block6 = BasicBlock(256,256)
        
        self.block7 = BasicBlock(256,512, stride=2) # 8x8
        self.skipdown3 = SkipDownSample(256,512, stride=2)            # MAYBE SHIFT THIS DOWN 1 SET OF BLOCKS
        self.block8 = BasicBlock(512,512)
        
        # I added
        self.block9 = BasicBlock(512, 1024) # 8x8
        self.skipdown4 = SkipDownSample(512,1024)
        
        self.block10 = BasicBlock(1024,2048) # 8x8
        self.skipdown5 = SkipDownSample(1024,2048)

        # EMBEDDING
        self.avgpool = AvgPool2d(kernel_size=8, stride=1) # kernel size 4 -> 8
        
        # CLASSIFICATION
        self.classification = Linear(2048, 4000)
    
    
    def forward(self, x):
        # HEAD
        # 64x64
        out = self.head(x)
        # 32x32
        # EMBEDDING
        out = self.block1(out) + out
        out = self.block2(out) + out
        out = self.block3(out) + self.skipdown1(out) # 16x16
        out = self.block4(out) + out
        out = self.block5(out) + self.skipdown2(out)# 8x8
        out = self.block6(out) + out
        out = self.block7(out) + self.skipdown3(out)# 4x4
        out = self.block8(out) + out
        out = self.block9(out) + self.skipdown4(out)# 2x2
        out = self.block10(out) + self.skipdown5(out)
        
        out = self.avgpool(out)
        embedding = torch.flatten(out,1)
        
        classification = self.classification(embedding)
                
        return embedding, classification

In [4]:
class ContrastiveLoss(Module):
    """
    Args:
        margin: 
    """
    def __init__(self, margin=1.):
        super().__init__()
        self.margin = margin

    def forward(self, x1, x2, labels):
        """
        Args:
            x: feature matrix with shape (batch_size, feat_dim).
            labels: ground truth labels with shape (batch_size).
        """
#         batch_size = x1.shape[0]
#         dist = torch.zeros(batch_size)
#         for i in range(batch_size):
#             dist[i] = torch.norm(x1[i]-x2[i])
        
        dist = CosineSimilarity(dim=1)(x1, x2)
    
        #total_loss = labels*dist + (1-labels)*(self.margin-dist)
        total_loss = (1-labels)*dist + labels*(self.margin-dist)

        loss = total_loss.mean()

        return loss

In [14]:
device = torch.device("cuda" if cuda else "cpu")

model_path = 'hw2p2_models/model_26.3_7/model.pt'
model = torch.load(model_path).to(device)
# model = Model().to(device)

RUN_NUMBER = 26.4  # <============================= CHANGE THIS EVERY TIME ======================<<<<<<<<<

NUM_EPOCHS = 15 # 40 #
learning_rate = 1e-6 # 1e-3 #1e-1
mile_stones = [5,10] # [5,10,15] # [4,7,10,13,16,19,22,25] #
gamma = 0.1
optimizer = optim.SGD(model.parameters(), momentum=0.9, weight_decay=5e-5, lr=learning_rate)

# TODO: TRY RMSProp
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=mile_stones, gamma=gamma)

criterion_ver = ContrastiveLoss()
criterion_class = CrossEntropyLoss()

In [7]:
L=ContrastiveLoss(device=device)
x1 = torch.Tensor([[1,1],[2,2],[3,3]])
x2 = torch.Tensor([[2,2],[4,4],[6,6]])
y=torch.Tensor([1,0,1])
L(x1,x2,y)

tensor(0.6667)

In [12]:
AUC = verify(model, ver_val_loader, 'none')
print("Validation AUC:", AUC)

Validation AUC: 0.9254600541437519


In [15]:
# toggle to train so save gradients
# running_max = ['',0.]
# while running_max[1] < 0.98:

val_accuracies=[]
model_number=0
prev_auc = 0
running_max = ['',0.]
for epoch in range(NUM_EPOCHS):
    ti = time()
    model.train()
    for i, (x, y, match) in enumerate(ver_train_loader):
        optimizer.zero_grad()

        img1 = x[0].to(device)
        img2 = x[1].to(device)
#         y = y.reshape(-1).to(device) # need to turn to row

        embedding1, out1 = model(img1)
        embedding2, out2 = model(img2)
        
        loss = criterion_ver(embedding1, embedding2, match.to(device))# + criterion_class(out1, y[0].to(device)) + criterion_class(out2, y[1].to(device))# + 
        loss.backward()
        
#         loss = criterion_class(out1, y[0])
#         loss.backward()
#         loss = criterion_class(out2, y[1])
#         loss.backward()

        optimizer.step()

        # progress
        if i%10==0:
            print('Epoch:', epoch, '| Iteration:', i, end='\r')

    
    # Deallocate memory in GPU
    torch.cuda.empty_cache()
    del x
    del y

    # validation
    accuracy = validate(model, val_loader, val_dataset)
    AUC = verify(model, ver_val_loader, sub_name)
    val_accuracies.append(accuracy)
    
    print("Epoch", epoch, "Accuracy:", accuracy, "AUC:", AUC, "Loss:", loss.item())
    if prev_auc == 0:
        print("\tImprovement:", AUC-prev_auc)
    else:
        print("\tImprovement:", AUC-prev_auc, "| Percent Improvement (times):", 100*AUC/prev_auc, '%')
    # tracking running best AUC
    if running_max[1]<AUC:
        running_max[0]='Model_' + str(RUN_NUMBER) + '_' + str(epoch)
        running_max[1]=AUC
    
    if (accuracy > 0.65 or AUC > 0.90):
        save_state(AUC, accuracy, running_max, model_number, model, ver_val_loader_args, device, NUM_EPOCHS, learning_rate, optimizer, criterion_class)
    model_number+=1

    scheduler.step()
        
    ver_train_dataset.load_pairs(100000, 0.5)
    
    tf=time()
    print("Time for epoch:", tf-ti)
    print('   Running Max:', *running_max,'\n')

    prev_auc = AUC

Epoch 0 Accuracy: 0.75625 AUC: 0.9239550230072234 Loss: 0.24474437534809113
	Improvement: 0.9239550230072234
Train Data Loaded: Total 100000 with 50000 matches and 50000 different
Time for epoch: 1187.6698563098907
   Running Max: Model_26.4_0 0.9239550230072234 



KeyboardInterrupt: 

In [9]:
# save_state(AUC, accuracy, running_max, model_number, model, ver_val_loader_args, device, NUM_EPOCHS, learning_rate, optimizer, criterion_class)
accuracy

0.69275

## Verification

In [1]:
import os
import numpy as np
import torch
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torch.nn import Module, CosineSimilarity, Linear, BatchNorm1d, BatchNorm2d, Sequential, Dropout, ReLU, LeakyReLU, Conv2d, MaxPool2d, AvgPool2d, AdaptiveAvgPool2d, CrossEntropyLoss

In [2]:
class VerificationMyDataset(Dataset):
    def __init__(self, pair_path, test=False):
        pairs = open(pair_path, "r").readlines(-1)
        self.test = test
        self.x=[]
        if not self.test:
            self.y=[]
        self.to_tensor = ToTensor()
        self.length=len(pairs)
        for i in range(self.length):
            line = pairs[i].strip().split(' ')
            image1 = self.to_tensor(Image.open(line[0]))
            image2 = self.to_tensor(Image.open(line[1]))
            self.x.append((image1,image2))
            if not self.test:
                self.y.append(int(line[2]))
        
    def __len__(self):
        return self.length

    # keep this simple/quick because run many times
    def __getitem__(self, index):
        if not self.test:
            return self.x[index], self.y[index]
        else:
            return self.x[index]

In [3]:
cuda = True
batchsize = 128 if cuda else 64
num_workers = 4 if cuda else 0
device = torch.device("cuda" if cuda else "cpu")
# from Recitation 5
pairs_path = "verification_pairs_val.txt"

ver_val_dataset = VerificationMyDataset(pairs_path)
ver_val_loader_args = dict(batch_size=batchsize, shuffle=False, num_workers=num_workers, pin_memory=True) if cuda else dict(shuffle=False, batch_size=batchsize)
ver_val_loader = DataLoader(ver_val_dataset, **ver_val_loader_args)

In [7]:
# RESNET BASIC BLOCK
class BasicBlock(Module): 
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, dropout=False, drop_rate=0.3): # groups=1, dilation=1, norm_layer=None
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        layers = [
            Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=kernel_size-2, stride=stride, bias=False),  # , padding=kernel_size-1
            BatchNorm2d(out_channels),
            ReLU(),
            Conv2d(out_channels, out_channels, kernel_size=kernel_size, padding=kernel_size-2, stride=1, bias=False),  # , padding=kernel_size-1
            BatchNorm2d(out_channels),
            ReLU()
        ]
        
        if dropout:
            layers.append(Dropout(drop_rate))
            
        self.block = Sequential(*layers)

    def forward(self,x):
        out = self.block(x)
         
        return out

class SkipDownSample(Module):
    def __init__(self, in_channels, out_channels, stride=1): # groups=1, dilation=1, norm_layer=None
        super().__init__()
    
        self.skip = Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)

    
    def forward(self, x):
        return self.skip(x)
    
# Model_26_ 
class Model(Module):
    def __init__(self):
        super().__init__()
        
        # HEAD
        head = [
            Conv2d(3, 64, kernel_size=3, stride=1, bias=False), # 64x64
            MaxPool2d(kernel_size=3, stride=2, padding=1) # 32x32
        ]
        self.head = Sequential(*head)
        
        # Embedder
        self.block1 = BasicBlock(64,64)
        self.block2 = BasicBlock(64,64)
        
        self.block3 = Sequential(BasicBlock(64,128), MaxPool2d(kernel_size=3, stride=2, padding=1))#, stride=2) # 16x16
        self.skipdown1 = SkipDownSample(64,128, stride=2)             # MAYBE SHIFT THIS DOWN 1 SET OF BLOCKS
        self.block4 = BasicBlock(128,128)
        
        self.block5 = BasicBlock(128,256) # , stride=2) # 16x16
        self.skipdown2 = SkipDownSample(128,256) # , stride=2)
        self.block6 = BasicBlock(256,256)
        
        self.block7 = BasicBlock(256,512, stride=2) # 8x8
        self.skipdown3 = SkipDownSample(256,512, stride=2)            # MAYBE SHIFT THIS DOWN 1 SET OF BLOCKS
        self.block8 = BasicBlock(512,512)
        
        # I added
        self.block9 = BasicBlock(512, 1024) # 8x8
        self.skipdown4 = SkipDownSample(512,1024)
        
        self.block10 = BasicBlock(1024,2048) # 8x8
        self.skipdown5 = SkipDownSample(1024,2048)

        # EMBEDDING
        self.avgpool = AvgPool2d(kernel_size=8, stride=1) # kernel size 4 -> 8
        
        # CLASSIFICATION
        self.classification = Linear(2048, 4000)
    
    
    def forward(self, x):
        # HEAD
        # 64x64
        out = self.head(x)
        # 32x32
        # EMBEDDING
        out = self.block1(out) + out
        out = self.block2(out) + out
        out = self.block3(out) + self.skipdown1(out) # 16x16
        out = self.block4(out) + out
        out = self.block5(out) + self.skipdown2(out)# 8x8
        out = self.block6(out) + out
        out = self.block7(out) + self.skipdown3(out)# 4x4
        out = self.block8(out) + out
        out = self.block9(out) + self.skipdown4(out)# 2x2
        out = self.block10(out) + self.skipdown5(out)
        
        out = self.avgpool(out)
        embedding = torch.flatten(out,1)
        
        classification = self.classification(embedding)
                
        return embedding#, classification

model_path = 'hw2p2_models/model_26.3_7/model.pt' # 25_ for submission 4
model = torch.load(model_path).to(device)
print(model)

Model(
  (head): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (block1): BasicBlock(
    (block): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
    )
  )
  (block2): BasicBlock(
    (block): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d

In [8]:
import csv
from sklearn.metrics import roc_auc_score

def verify(model, data_loader, sub_name, test=False):
    model.eval()
    
    cos_sim = CosineSimilarity(dim=1)
    cosine_similarity = torch.Tensor([])
    true_similarity = torch.Tensor([])
    if not test:
        for i, (x,y) in enumerate(data_loader):
            img1 = x[0].to(device)
            img2 = x[1].to(device)
            y = y.to("cpu")
            
            out1 = model(img1).to("cpu")
            out2 = model(img2).to("cpu")
            
            cosine_similarity = torch.cat((cosine_similarity.detach(), cos_sim(out1,out2).detach()), 0)
            true_similarity = torch.cat((true_similarity, y), 0)

            del x, y, img1, img2, out1, out2
            torch.cuda.empty_cache()
            if i%10==0:
                print("Verification on validation set:", i*batchsize, end='\r')
            
        AUC = roc_auc_score(true_similarity, cosine_similarity.detach().numpy())
        return AUC
    else:
        for i, (x) in enumerate(data_loader):
            img1 = x[0].to(device)
            img2 = x[1].to(device)
            
            out1 = model(img1).to("cpu")
            out2 = model(img2).to("cpu")
            
            cosine_similarity = torch.cat((cosine_similarity.detach(), cos_sim(out1,out2).detach()), 0)
            if i%10==0:
                print("Verification on test set:", i*batchsize, end='\r')
        return write_submission(sub_name, cosine_similarity)

def write_submission(sub_name, cosine_similarity):
    pair_path = "verification_pairs_test.txt"
    pairs = open(pair_path, "r").readlines(-1)
    
    submission = csv.writer(open(sub_name, "w"))
    submission.writerow(['Id','Category'])

    self_length=len(pairs)
    for i in range(self_length):
        submission.writerow([pairs[i].strip(),cosine_similarity[i].item()])
        if i%10==0:
            print("Saved {} predicitons".format((i+1)*batchsize), end='\r')

    print("--- Submission File COMPLETE ---")
    return self_length

# sub_name = "DELETEME"
# AUC = verify(model, ver_val_loader, sub_name)
# print("Validation AUC:", AUC)        

Validation AUC: 0.925460338024243380


In [9]:
sub_name = './submission6.csv'

pairs_path = "verification_pairs_test.txt"

test_dataset = VerificationMyDataset(pairs_path, test=True)
test_loader_args = dict(batch_size=batchsize, shuffle=False, num_workers=num_workers, pin_memory=True) if cuda else dict(shuffle=False, batch_size=batchsize)
test_loader = DataLoader(test_dataset, **test_loader_args)
print("Data Loaded")

T = verify(model, test_loader, sub_name, test=True)
print("{} Predictions COMPLETE".format(T))

Data Loaded
--- Submission File COMPLETE ---
51835 Predictions COMPLETE


## Beginnings of Success ---------------------------------------------------------------------------------------

In [ ]:
# RESNET BASIC BLOCK
# Model_27_ 
class Model(Module):
    def __init__(self):
        super().__init__()
        
        # HEAD
        head = [
            Conv2d(3, 64, kernel_size=3, stride=1, bias=False), # 64x64
            MaxPool2d(kernel_size=3, stride=2, padding=1) # 32x32
        ]
        self.head = Sequential(*head)
        
        # Embedder
        self.block1 = BasicBlock(64,64)
        self.block2 = BasicBlock(64,64)
        
        self.block3 = Sequential(BasicBlock(64,128), MaxPool2d(kernel_size=3, stride=2, padding=1)) #, stride=2) # 16x16
        self.skipdown1 = SkipDownSample(64,128, stride=2)             # MAYBE SHIFT THIS DOWN 1 SET OF BLOCKS
        self.block4 = BasicBlock(128,128)
        
        self.block5 = BasicBlock(128,256) # , stride=2) # 16x16
        self.skipdown2 = SkipDownSample(128,256) # , stride=2)
        self.block6 = BasicBlock(256,256)
        
        self.block7 = Sequential(BasicBlock(256,512), MaxPool2d(kernel_size=3, stride=2, padding=1)) # , stride=2) # 8x8             # MAYBE MAKE THIS MAX POOL
        self.skipdown3 = SkipDownSample(256,512, stride=2)
        self.block8 = BasicBlock(512,512)
        
        # I added
        self.block9 = BasicBlock(512, 1024) # 8x8
        self.skipdown4 = SkipDownSample(512,1024)
        
        self.block10 = BasicBlock(1024,2048) # 8x8
        self.skipdown5 = SkipDownSample(1024,2048)

        # EMBEDDING
        self.avgpool = AvgPool2d(kernel_size=8, stride=1) # kernel size 4 -> 8
        
        # CLASSIFICATION
        self.classification = Linear(2048, 4000)
        #             Linear(1024, 4000)

        
#         ]
#         self.classification = Sequential(*classification)
    
    
    def forward(self, x):
        # HEAD
        # 64x64
        out = self.head(x)
        # 32x32
        # EMBEDDING
        out = self.block1(out) + out
        out = self.block2(out) + out
        out = self.block3(out) + self.skipdown1(out) # 16x16
        out = self.block4(out) + out
        out = self.block5(out) + self.skipdown2(out)# 8x8
        out = self.block6(out) + out
        out = self.block7(out) + self.skipdown3(out)# 4x4
        out = self.block8(out) + out
        out = self.block9(out) + self.skipdown4(out)# 2x2
        out = self.block10(out) + self.skipdown5(out)
        
        out = self.avgpool(out)
        embedding = torch.flatten(out,1)
        
        classification = self.classification(embedding)
                
        return embedding, classification

# Model_26_ 
class Model(Module):
    def __init__(self):
        super().__init__()
        
        # HEAD
        head = [
            Conv2d(3, 64, kernel_size=3, stride=1, bias=False), # 64x64
            MaxPool2d(kernel_size=3, stride=2, padding=1) # 32x32
        ]
        self.head = Sequential(*head)
        
        # Embedder
        self.block1 = BasicBlock(64,64)
        self.block2 = BasicBlock(64,64)
        
        self.block3 = Sequential(BasicBlock(64,128), MaxPool2d(kernel_size=3, stride=2, padding=1))#, stride=2) # 16x16
        self.skipdown1 = SkipDownSample(64,128, stride=2)             # MAYBE SHIFT THIS DOWN 1 SET OF BLOCKS
        self.block4 = BasicBlock(128,128)
        
        self.block5 = BasicBlock(128,256) # , stride=2) # 16x16
        self.skipdown2 = SkipDownSample(128,256) # , stride=2)
        self.block6 = BasicBlock(256,256)
        
        self.block7 = BasicBlock(256,512, stride=2) # 8x8
        self.skipdown3 = SkipDownSample(256,512, stride=2)            # MAYBE SHIFT THIS DOWN 1 SET OF BLOCKS
        self.block8 = BasicBlock(512,512)
        
        # I added
        self.block9 = BasicBlock(512, 1024) # 8x8
        self.skipdown4 = SkipDownSample(512,1024)
        
        self.block10 = BasicBlock(1024,2048) # 8x8
        self.skipdown5 = SkipDownSample(1024,2048)

        # EMBEDDING
        self.avgpool = AvgPool2d(kernel_size=8, stride=1) # kernel size 4 -> 8
        
        # CLASSIFICATION
        self.classification = Linear(2048, 4000)
    
    
    def forward(self, x):
        # HEAD
        # 64x64
        out = self.head(x)
        # 32x32
        # EMBEDDING
        out = self.block1(out) + out
        out = self.block2(out) + out
        out = self.block3(out) + self.skipdown1(out) # 16x16
        out = self.block4(out) + out
        out = self.block5(out) + self.skipdown2(out)# 8x8
        out = self.block6(out) + out
        out = self.block7(out) + self.skipdown3(out)# 4x4
        out = self.block8(out) + out
        out = self.block9(out) + self.skipdown4(out)# 2x2
        out = self.block10(out) + self.skipdown5(out)
        
        out = self.avgpool(out)
        embedding = torch.flatten(out,1)
        
        classification = self.classification(embedding)
                
        return embedding, classification
    
# Model_25_ 
class Model(Module):
    def __init__(self):
        super().__init__()
        
        # HEAD
        head = [
            Conv2d(3, 64, kernel_size=3, stride=1, bias=False), # 64x64
            MaxPool2d(kernel_size=3, stride=2, padding=1) # 32x32
        ]
        self.head = Sequential(*head)
        
        # Embedder
        self.block1 = BasicBlock(64,64)
        self.block2 = BasicBlock(64,64)
        
        self.block3 = BasicBlock(64,128) # , stride=2) # 32x32
        self.skipdown1 = SkipDownSample(64,128) # , stride=2)             # MAYBE SHIFT THIS DOWN 1 SET OF BLOCKS
        self.block4 = BasicBlock(128,128)
        
        self.block5 = BasicBlock(128,256, stride=2) # 16x16
        self.skipdown2 = SkipDownSample(128,256, stride=2)
        self.block6 = BasicBlock(256,256)
        
        self.block7 = BasicBlock(256,512, stride=2) # 8x8
        self.skipdown3 = SkipDownSample(256,512, stride=2)            # MAYBE SHIFT THIS DOWN 1 SET OF BLOCKS
        self.block8 = BasicBlock(512,512)
        
        # I added
        self.block9 = BasicBlock(512, 1024) # 8x8
        self.skipdown4 = SkipDownSample(512,1024)
        
        self.block10 = BasicBlock(1024, 2048) # 8x8
        self.skipdown5 = SkipDownSample(1024,2048)

        # EMBEDDING
        self.avgpool = AvgPool2d(kernel_size=8, stride=1) # kernel size 4 -> 8
        
        # CLASSIFICATION
        self.classification = Linear(2048, 4000)
        #             Linear(1024, 4000)

        
#         ]
#         self.classification = Sequential(*classification)
    
    
    def forward(self, x):
        # HEAD
        # 64x64
        out = self.head(x)
        # 32x32
        # EMBEDDING
        out = self.block1(out) + out
        out = self.block2(out) + out
        out = self.block3(out) + self.skipdown1(out) # 16x16
        out = self.block4(out) + out
        out = self.block5(out) + self.skipdown2(out)# 8x8
        out = self.block6(out) + out
        out = self.block7(out) + self.skipdown3(out)# 4x4
        out = self.block8(out) + out
        out = self.block9(out) + self.skipdown4(out)# 2x2
        out = self.block10(out) + self.skipdown5(out)
        
        out = self.avgpool(out)
        embedding = torch.flatten(out,1)
        
        classification = self.classification(embedding)
                
        return embedding, classification
    
    
# Model_24_ 
class Model(Module):
    def __init__(self):
        super().__init__()
        
        # HEAD
        head = [
            Conv2d(3, 64, kernel_size=3, stride=1, bias=False), # 64x64
            MaxPool2d(kernel_size=3, stride=2, padding=1) # 32x32
        ]
        self.head = Sequential(*head)
        
        # Embedder
        self.block1 = BasicBlock(64,64)
        self.block2 = BasicBlock(64,64)
        
        self.block3 = BasicBlock(64,128, stride=2) # 16x16
        self.skipdown1 = SkipDownSample(64,128, stride=2)             # MAYBE SHIFT THIS DOWN 1 SET OF BLOCKS
        self.block4 = BasicBlock(128,128)
        
        self.block5 = BasicBlock(128,256) # , stride=2) # 16x16
        self.skipdown2 = SkipDownSample(128,256) # , stride=2)
        self.block6 = BasicBlock(256,256)
        
        self.block7 = BasicBlock(256,512, stride=2) # 8x8
        self.skipdown3 = SkipDownSample(256,512, stride=2)            # MAYBE SHIFT THIS DOWN 1 SET OF BLOCKS
        self.block8 = BasicBlock(512,512)
        
        # I added
        self.block9 = BasicBlock(512, 1024) # 8x8
        self.skipdown4 = SkipDownSample(512,1024)
        
        self.block10 = BasicBlock(1024, 2048) # 8x8
        self.skipdown5 = SkipDownSample(1024,2048)

        # EMBEDDING
        self.avgpool = AvgPool2d(kernel_size=8, stride=1) # kernel size 4 -> 8
        
        # CLASSIFICATION
        self.classification = Linear(2048, 4000)
        #             Linear(1024, 4000)

        
#         ]
#         self.classification = Sequential(*classification)
    
    
    def forward(self, x):
        # HEAD
        # 64x64
        out = self.head(x)
        # 32x32
        # EMBEDDING
        out = self.block1(out) + out
        out = self.block2(out) + out
        out = self.block3(out) + self.skipdown1(out) # 16x16
        out = self.block4(out) + out
        out = self.block5(out) + self.skipdown2(out)# 8x8
        out = self.block6(out) + out
        out = self.block7(out) + self.skipdown3(out)# 4x4
        out = self.block8(out) + out
        out = self.block9(out) + self.skipdown4(out)# 2x2
        out = self.block10(out) + self.skipdown5(out)
        
        out = self.avgpool(out)
        embedding = torch.flatten(out,1)
        
        classification = self.classification(embedding)
                
        return embedding, classification

In [ ]:
# Model_2_
class BasicBlock(Module): 
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, dropout=False, drop_rate=0.3): # groups=1, dilation=1, norm_layer=None
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        layers = [
            Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=kernel_size-2, stride=stride, bias=False),  # , padding=kernel_size-1
            BatchNorm2d(out_channels),
            ReLU(),
        ]

        if dropout:
            layers.append(Dropout(drop_rate))
            
        self.block = Sequential(*layers)
        
#         # Add padding to down sample
#         skip_layers = [
#             Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False),
#             BatchNorm2d(out_channels)
#         ]
#         self.skip = Sequential(*skip_layers)
        
#         self.relu = ReLU()

    def forward(self,x):
#         identity = x
        
        out = self.block(x)
         
#         identity = self.skip(identity)
#         out += identity

#         return self.relu(out)
        return out
# based on baseline
class Model(Module):
    def __init__(self):
        super().__init__()

#         BigBlock = []
#         for i in range(5):
#             BigBlock.append(DepthWiseBlock(512, 512))
#             BigBlock.append(BasicBlock(512, 512))
        
        # Embedder
        layers = [
            BasicBlock(3, 64, dropout=True),
            BasicBlock(64, 64),
            MaxPool2d(2,2),
            BasicBlock(64, 64),
            MaxPool2d(2,2),
            BasicBlock(64, 128, dropout=True),
            BasicBlock(128, 128),
            MaxPool2d(2,2),
            BasicBlock(128, 256, dropout=True),
            BasicBlock(256, 256),
            MaxPool2d(2,2),
            BasicBlock(256, 256),
            MaxPool2d(2,2),
            BasicBlock(256, 256),
            MaxPool2d(2,2)
        ]
        
        self.layers = Sequential(*layers)
        
        # EMBEDDING
        self.embed = Linear(256, 1024)
        
        # CLASSIFICATION
        classification = [
#             Linear(1024, 1024),          # TODO: Is this okay?
#             BatchNorm1d(1024),
#             ReLU(),
            Linear(1024, 4000)
        ]
        self.classification = Sequential(*classification)
    
    
    def forward(self, x):
        # HEAD
#         print(x.shape)
#         out = self.head(x)
#         print(out.shape)
        # LAYERS
        out = self.layers(x)
        
        # EMBEDDING
#         print(out.shape)
        out = torch.flatten(out,1)
        embedding = self.embed(out)
        
        classification = self.classification(embedding)
        
        return embedding, classification

In [ ]:
# Model_3_, Model_7_, Model_8_, Model_9_, Model_10_
class BasicBlock(Module): 
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, dropout=False, drop_rate=0.3): # groups=1, dilation=1, norm_layer=None
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        layers = [
            Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=kernel_size-2, stride=stride, bias=False),  # , padding=kernel_size-1
            BatchNorm2d(out_channels),
            ReLU()
        ]
        
        if dropout:
            layers.append(Dropout(drop_rate))
            
        self.block = Sequential(*layers)

    def forward(self,x):
        out = self.block(x)
         
        return out
class Model(Module):
    def __init__(self):
        super().__init__()

        # Embedder
        layers = [
            BasicBlock(3, 64, dropout=True),
            BasicBlock(64, 64),
            MaxPool2d(2,2),
            BasicBlock(64, 64),
            MaxPool2d(2,2),
            BasicBlock(64, 128, dropout=True),
            BasicBlock(128, 128),
            MaxPool2d(2,2),
            BasicBlock(128, 256, dropout=True),
            BasicBlock(256, 256),
            MaxPool2d(2,2),
            BasicBlock(256, 512),
            MaxPool2d(2,2),
            BasicBlock(512, 1024),
            MaxPool2d(2,2)
        ]
        
        # EMBEDDING
        self.embed = Sequential(*layers)
#         self.embed = Linear(256, 1000)

        
        # CLASSIFICATION
        self.classification = Linear(1024, 4000)
        #             Linear(1024, 4000)

        
#         ]
#         self.classification = Sequential(*classification)
    
    
    def forward(self, x):
        # HEAD
#         print(x.shape)
#         out = self.head(x)
#         print(out.shape)

        # EMBEDDING
        out = self.embed(x)
        embedding = torch.flatten(out,1)

#         print(embedding.shape)
    
        classification = self.classification(embedding)
                
        return embedding, classification
model = Model()
print(model)

In [ ]:
# Model_4_
class BasicBlock(Module): 
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, dropout=False, drop_rate=0.3, skip=False): # groups=1, dilation=1, norm_layer=None
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        layers = [
            Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=kernel_size-2, stride=stride, bias=False),  # , padding=kernel_size-1
            BatchNorm2d(out_channels),
            ReLU()
        ]
            
        self.block = Sequential(*layers)
        
        self.skip = skip
        if self.skip:
            skip_layers = [
                Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False),
                BatchNorm2d(out_channels)
            ]
            self.skip_output = Sequential(*skip_layers)
        
        self.relu = ReLU()
        
        if dropout:
            self.dropout=Dropout(drop_rate)
        else:
            self.dropout=None
            
    def forward(self,x):
        identity = x
        
        out = self.block(x)
        
        if self.skip:
            identity = self.skip_output(identity)
            out += identity

        out = self.relu(out)
        
        if self.dropout:
            out = self.dropout(out)
        
        return out

class Model(Module):
    def __init__(self):
        super().__init__()

        # Embedder
        layers = [
            BasicBlock(3, 64, dropout=True),
            BasicBlock(64, 64),
            MaxPool2d(2,2),
            BasicBlock(64, 64),
            MaxPool2d(2,2),
            BasicBlock(64, 128, dropout=True),
            BasicBlock(128, 128),
            MaxPool2d(2,2),
            BasicBlock(128, 256, dropout=True),
            BasicBlock(256, 256),
            MaxPool2d(2,2),
            BasicBlock(256, 512),
            MaxPool2d(2,2),
            BasicBlock(512, 1024),
            MaxPool2d(2,2)
        ]
        
        # EMBEDDING
        self.embed = Sequential(*layers)
        
        # CLASSIFICATION
        self.classification = Linear(1024, 4000)
    
    def forward(self, x):
        # EMBEDDING
        out = self.embed(x)
        embedding = torch.flatten(out,1)

        classification = self.classification(embedding)
                
        return embedding, classification

In [ ]:
# Model_6_
class BasicBlock(Module): 
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, dropout=False, drop_rate=0.3, skip=False): # groups=1, dilation=1, norm_layer=None
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        layers = [
            Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=kernel_size-2, stride=stride, bias=False),  # , padding=kernel_size-1
            BatchNorm2d(out_channels),
            LeakyReLU(),
            Conv2d(out_channels, out_channels, kernel_size=kernel_size, padding=kernel_size-2, stride=stride, bias=False),  # , padding=kernel_size-1
            BatchNorm2d(out_channels),
            LeakyReLU()
        ]
            
        self.block = Sequential(*layers)
        
        # Add padding to down sample
        
        self.skip = skip
        if self.skip:
            skip_layers = [
                Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False),
                BatchNorm2d(out_channels)
            ]
            self.skip_output = Sequential(*skip_layers)
        
        self.leakyrelu = LeakyReLU()
        
        if dropout:
            self.dropout=Dropout(drop_rate)
        else:
            self.dropout=None
            
    def forward(self,x):
        connection = x
        
        out = self.block(x)
        
        if self.skip:
            connection = self.skip_output(connection)
            out += connection

        out = self.leakyrelu(out)
        
        if self.dropout:
            out = self.dropout(out)
        
        return out

# ResNet Influence, dropout from baseline
class Model(Module):
    def __init__(self):
        super().__init__()
        
        # BIG BLOCK
        layers = [
            # HEAD
            Conv2d(3, 64, kernel_size=7, padding=3, stride=1, bias=False),
            MaxPool2d(2,2),
            # BLOCK
            BasicBlock(64, 64, dropout=True, skip=True),
            BasicBlock(64, 64, skip=True),
            BasicBlock(64, 64, skip=True),
            MaxPool2d(2,2),
            BasicBlock(64, 128, dropout=True, skip=True),
            BasicBlock(128, 128, skip=True),
            BasicBlock(128, 128, skip=True),
            MaxPool2d(2,2),
            BasicBlock(128, 256, dropout=True, skip=True),
            BasicBlock(256, 256, skip=True),
            BasicBlock(256, 256, skip=True),
            BasicBlock(256, 256, skip=True),
            MaxPool2d(2,2),
            BasicBlock(256, 512, skip=True),
            BasicBlock(512, 512, skip=True),
            BasicBlock(512, 512, skip=True),
            MaxPool2d(2,2),
            BasicBlock(512, 1024, skip=True),
            BasicBlock(1024, 1024, skip=True),
            MaxPool2d(2,2)
        ]

        self.big_block = Sequential(*layers)

        # EMBEDDING
        self.embed = Linear(1024, 2048, bias=False)

        
        # CLASSIFICATION
        class_layers = [
            Linear(2048, 4096, bias=False),
            BatchNorm1d(4096),
            ReLU(),
            Linear(4096, 4000, bias=False)
        ]
        self.classification = Sequential(*class_layers)
        
    def forward(self, x):
        # BIG BLOCK
        out = self.big_block(x)
        
        out = torch.flatten(out,1)

        # EMBEDDING
        embedding = self.embed(out)

#         print(embedding.shape)
        # CLASSIFY
        classification = self.classification(embedding)
                
        return embedding, classification, out

In [ ]:
# Model_15_ MobileNet - 1 skip over big block
class BasicBlock(Module): 
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1): # groups=1, dilation=1, norm_layer=None
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        layers = [
            Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=kernel_size-2, stride=stride, bias=False),  # , padding=kernel_size-1
            BatchNorm2d(out_channels),
            ReLU(),
        ]

        self.block = Sequential(*layers)
        
    def forward(self,x):
        out = self.block(x)
         
        return out

class DepthWiseBlock(Module): 
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1): # groups=1, dilation=1, norm_layer=None
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        layers = [
            Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=kernel_size-2, stride=stride, bias=False, groups=in_channels),
            BatchNorm2d(out_channels),
            ReLU(),
            Conv2d(out_channels, out_channels, kernel_size=1, stride=1, bias=False),
            BatchNorm2d(out_channels),
            ReLU()
        ]

        self.block = Sequential(*layers)
        
    def forward(self,x):
        out = self.block(x)

        return out
# based on MobileNet
class Model(Module):
    def __init__(self):
        super().__init__()
        # Embedder
        layers1 = [
            BasicBlock(3, 32, stride=1), # originally stride=2, BasicBlock(3, 32, stride=1),
            DepthWiseBlock(32, 32),
            BasicBlock(32, 64),
            DepthWiseBlock(64, 64, stride=2),
            BasicBlock(64, 128),
            DepthWiseBlock(128, 128),
            BasicBlock(128, 128),
            DepthWiseBlock(128, 128, stride=2),
            BasicBlock(128, 256),
            DepthWiseBlock(256, 256),
            BasicBlock(256, 256),
            DepthWiseBlock(256, 256, stride=2),
            BasicBlock(256, 512)
        ]
        
        BigBlock = []
        for i in range(5):
            BigBlock.append(DepthWiseBlock(512, 512))
            BigBlock.append(BasicBlock(512, 512))
            
        layers2 = [
            DepthWiseBlock(512, 512, stride=2),
            BasicBlock(512, 1024),
            DepthWiseBlock(1024, 1024, stride=2),
            BasicBlock(1024, 1024)
        ]
        
        self.layers1 = Sequential(*layers1)
        self.BigBlock = Sequential(*BigBlock)
        self.layers2 = Sequential(*layers2)
        
        # EMBEDDING
        self.avgpool = AvgPool2d(2, stride=1)
        
        
        self.classification = Linear(1024, 4000)    
    
    def forward(self, x):
        # Layers1
        out = self.layers1(x)
        
        # Big Block
        out = self.BigBlock(out) + out # skip connection
        
        # layers2
        out = self.layers2(out)
        
        # EMBEDDING
        out = self.avgpool(out)
        embedding = torch.flatten(out,1)
        
        classification = self.classification(embedding)
        
        return embedding, classification

In [ ]:
# Model_17_ MobileNet2
class BasicBlock(Module): 
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1): # groups=1, dilation=1, norm_layer=None
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        layers = [
            Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=kernel_size-2, stride=stride, bias=False),  # , padding=kernel_size-1
            BatchNorm2d(out_channels),
            ReLU(),
        ]

        self.block = Sequential(*layers)
        
#         # Add padding to down sample
#         skip_layers = [
#             Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False),
#             BatchNorm2d(out_channels)
#         ]
#         self.skip = Sequential(*skip_layers)
        
#         self.relu = ReLU()

    def forward(self,x):
#         identity = x
        
        out = self.block(x)
         
#         identity = self.skip(identity)
#         out += identity

#         return self.relu(out)
        return out

class DepthWiseBlock(Module): 
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1): # groups=1, dilation=1, norm_layer=None
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        layers = [
            Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=kernel_size-2, stride=stride, bias=False, groups=in_channels),
            BatchNorm2d(out_channels),
            ReLU(),
            Conv2d(out_channels, out_channels, kernel_size=1, stride=1, bias=False),
            BatchNorm2d(out_channels),
            ReLU()
        ]

        self.block = Sequential(*layers)
        
    def forward(self,x):
        out = self.block(x)

        return out
# based on MobileNet
class Model(Module):
    def __init__(self):
        super().__init__()
        # Embedder
        layers1 = [
            BasicBlock(3, 32, stride=1), # originally stride=2, BasicBlock(3, 32, stride=1),
            DepthWiseBlock(32, 32),
            BasicBlock(32, 64),
            DepthWiseBlock(64, 64, stride=2),
            BasicBlock(64, 128),
            DepthWiseBlock(128, 128),
            BasicBlock(128, 128),
            DepthWiseBlock(128, 128, stride=2),
            BasicBlock(128, 256),
            DepthWiseBlock(256, 256),
            BasicBlock(256, 256),
            DepthWiseBlock(256, 256, stride=2),
            BasicBlock(256, 512)
        ]
        
        BigBlock = []
        for i in range(5):
            BigBlock.append(DepthWiseBlock(512, 512))
            BigBlock.append(BasicBlock(512, 512))
            
        layers2 = [
            DepthWiseBlock(512, 1024, stride=2),
            BasicBlock(1024, 1024),
            DepthWiseBlock(1024, 2048, stride=2),
            BasicBlock(2048, 2048)
        ]
        
        self.layers1 = Sequential(*layers1)
        self.BigBlock = Sequential(*BigBlock)
        self.layers2 = Sequential(*layers2)
        
        # EMBEDDING
        self.avgpool = AvgPool2d(2, stride=1)
        
#         classification_layers = [
#             Linear(1024, 2048),            # DO THIS AFTER 2048 embedding then can try contrastive loss after training this
#             BatchNorm1d(),
#             ReLU(),
#             Linear(2048, 4000)
#         ]
        self.classification = Linear(2048, 4000) # Sequential(*classification_layers) # 
    
    def forward(self, x):
        # Layers1
        out = self.layers1(x)
        
        # Big Block
        out = self.BigBlock(out) + out # skip connection
        
        # layers2
        out = self.layers2(out)
        
        # EMBEDDING
        out = self.avgpool(out)
        embedding = torch.flatten(out,1)
        
        classification = self.classification(embedding)
        
        return embedding, classification

## Needs Work

In [ ]:
# Model_1_ MobileNet
class BasicBlock(Module): 
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1): # groups=1, dilation=1, norm_layer=None
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        layers = [
            Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=kernel_size-2, stride=stride, bias=False),  # , padding=kernel_size-1
            BatchNorm2d(out_channels),
            ReLU(),
        ]

        self.block = Sequential(*layers)
        
#         # Add padding to down sample
#         skip_layers = [
#             Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False),
#             BatchNorm2d(out_channels)
#         ]
#         self.skip = Sequential(*skip_layers)
        
#         self.relu = ReLU()

    def forward(self,x):
#         identity = x
        
        out = self.block(x)
         
#         identity = self.skip(identity)
#         out += identity

#         return self.relu(out)
        return out

class DepthWiseBlock(Module): 
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1): # groups=1, dilation=1, norm_layer=None
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        layers = [
            Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=kernel_size-2, stride=stride, bias=False, groups=in_channels),
            BatchNorm2d(out_channels),
            ReLU(),
            Conv2d(out_channels, out_channels, kernel_size=1, stride=1, bias=False),
            BatchNorm2d(out_channels),
            ReLU()
        ]

        self.block = Sequential(*layers)
        
        # Add padding to down sample
#         skip_layers = [
#             Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False),
#             BatchNorm2d(out_channels)
#         ]
#         self.skip = Sequential(*skip_layers)
        
#         self.relu = ReLU()

    def forward(self,x):
#         identity = x
        
        out = self.block(x)
         
#         identity = self.skip(identity)
#         out += identity

#         return self.relu(out)
        return out

# based on MobileNet
class Model(Module):
    def __init__(self):
        super().__init__()
        
        # HEAD: maybe have a head with max pooling
        head = [
            Conv2d(3, 32, kernel_size=3, padding=1, bias=False),
            BatchNorm2d(32),
            ReLU(),
            Dropout()
        ]
        self.head = Sequential(*head)
        
        BigBlock = []
        for i in range(5):
            BigBlock.append(DepthWiseBlock(512, 512))
            BigBlock.append(BasicBlock(512, 512))
        
        # Embedder
        layers = [
            BasicBlock(32, 32, stride=1), # originally stride=2, BasicBlock(3, 32, stride=1),
            DepthWiseBlock(32, 32),
            BasicBlock(32, 64),
            DepthWiseBlock(64, 64, stride=2),
            BasicBlock(64, 128),
            DepthWiseBlock(128, 128),
            BasicBlock(128, 128),
            DepthWiseBlock(128, 128, stride=2),
            BasicBlock(128, 256),
            DepthWiseBlock(256, 256),
            BasicBlock(256, 256),
            DepthWiseBlock(256, 256, stride=2),
            BasicBlock(256, 512),
            *BigBlock,
            DepthWiseBlock(512, 512, stride=2),
            BasicBlock(512, 1024),
            DepthWiseBlock(1024, 1024, stride=2),
            BasicBlock(1024, 1024)
#             AvgPool2d()
        ]
        
        self.layers = Sequential(*layers)
        
        # EMBEDDING
        self.avgpool = AvgPool2d(2, stride=1)  # TODO: Check if this works
        self.embed = Linear(1024, 1024)
        
        # CLASSIFICATION
        classification = [
            Linear(1024, 1024),          # TODO: Is this okay?
            BatchNorm1d(1024),
            ReLU(),
            Linear(1024, 4000)
        ]
        self.classification = Sequential(*classification)
    
    
    def forward(self, x):
        # Layers
        out = self.layers(x)
        
        # EMBEDDING
        out = self.avgpool(out)
        out = torch.flatten(out,1)
        embedding = self.embed(out)
        
        classification = self.classification(embedding)
        
        return embedding, classification

In [ ]:
# based on code from: https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py
class BasicBlock(Module): 
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1): # groups=1, dilation=1, norm_layer=None
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        layers = [
            Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=kernel_size-2, stride=stride, bias=False),  # , padding=kernel_size-1
            BatchNorm2d(out_channels),
            ReLU(),
            Conv2d(out_channels, out_channels, kernel_size=kernel_size, padding=kernel_size-2, stride=stride, bias=False), # , padding=kernel_size-1
            BatchNorm2d(out_channels)
        ]

        self.block_preskip = Sequential(*layers)
        
        # Used for downsampling or updating x to be in same form/size as convolution
        # Add padding to down sample
        skip_layers = [
            Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False),
            BatchNorm2d(out_channels)
        ]
        self.skip = Sequential(*skip_layers)
        
        self.relu = ReLU()

    def forward(self,x):
        identity = x
        
        out = self.block_preskip(x)
         
        # TODO: implement skip connections - TEST INDIVIDUALLY WITH FORWARD ON RANDOM INPUT
#         if downsample is not None:
#         if self.in_channels != self.out_channels:
        identity = self.skip(identity)
#     torch.Size([32, 128, 63, 63]) torch.Size([32, 128, 59, 59]) torch.Size([32, 64, 59, 59])
        print(out.shape, identity.shape, x.shape)
        out += identity

        return self.relu(out)
    
# based on ResNet from: https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py
class Model(Module):
    def __init__(self):
        super().__init__()
        
        # HEAD
        head = [
            Conv2d(3, 64, kernel_size=8, stride=1, padding=1, bias=False),
            BatchNorm2d(64),
            ReLU(),
            MaxPool2d(kernel_size=3, stride=1, padding=1)
        ]
        self.head = Sequential(*head)
        
        # BASIC BLOCKS
#         block = []
        self.block1 = self._make_block(2, 64, 128)
        self.block2 = self._make_block(2, 128, 256)
        self.block3 = self._make_block(2, 256, 512)
        self.block4 = self._make_block(2, 512, 512)
        
        # TAIL
        self.avgpool = AdaptiveAvgPool2d((1,1))  # TODO: Check if this works
        self.embed = Linear(512, 1024)
        
        # CLASSIFICATION
        classification = [
            Linear(1024, 1024),          # TODO: Is this okay?
            BatchNorm1d(1024),
            ReLU(),
            Linear(1024, 4000)
        ]
        self.classification = Sequential(*classification)
    
    def _make_block(self, blocks, in_channel, out_channel, kernel_size=3, stride=1):
        layers = []
        layers.append(BasicBlock(in_channel, out_channel, kernel_size=kernel_size, stride=stride))
        for i in range(1,blocks):
            layers.append(BasicBlock(out_channel, out_channel, kernel_size=kernel_size, stride=stride))
         
        return Sequential(*layers)
    
    def forward(self, x):
        out = self.head(x)
        
        out = self.block1(out)
        out = self.block2(out)
        out = self.block3(out)
        out = self.block4(out)
        
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        embedding = self.embed(out)
        
        classification = self.classification(embedding)
        
        return embedding, classification


In [ ]:
# Model_7_, Model_10_
class BasicBlock(Module): 
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, dropout=False, drop_rate=0.3): # groups=1, dilation=1, norm_layer=None
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        layers = [
            Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=kernel_size-2, stride=stride, bias=False),  # , padding=kernel_size-1
            BatchNorm2d(out_channels),
            ReLU()
        ]
        
        if dropout:
            layers.append(Dropout(drop_rate))
            
        self.block = Sequential(*layers)

    def forward(self,x):
        out = self.block(x)
         
        return out

# Model_7_, Model_10_
class Model(Module):
    def __init__(self):
        super().__init__()

        # Embedder
        layers = [
            BasicBlock(3, 64, dropout=True),
            BasicBlock(64, 64),
            MaxPool2d(2,2),
            BasicBlock(64, 64, dropout=True),
            BasicBlock(64, 64),
            MaxPool2d(2,2),
            BasicBlock(64, 128, dropout=True),
            BasicBlock(128, 128),
            MaxPool2d(2,2),
            BasicBlock(128, 256),
            BasicBlock(256, 256),
            BasicBlock(256, 256),
            BasicBlock(256, 256),
            MaxPool2d(2,2),
            BasicBlock(256, 512),
            MaxPool2d(2,2),
            BasicBlock(512, 1024),
            MaxPool2d(2,2)
        ]
        
        # EMBEDDING
        self.embed = Sequential(*layers)
#         self.embed = Linear(256, 1000)

        
        # CLASSIFICATION
        self.classification = Linear(1024, 4000)
        #             Linear(1024, 4000)

        
#         ]
#         self.classification = Sequential(*classification)
    
    
    def forward(self, x):
        # EMBEDDING
        out = self.embed(x)
        embedding = torch.flatten(out,1)
    
        classification = self.classification(embedding)
                
        return embedding, classification

In [ ]:
# can't really make siamese models like this because the backwards will not be identical
val_accuracies=[]
model_number=0
prev_auc = 0
running_max = ['',0.]
for epoch in range(NUM_EPOCHS):
    ti = time()
    model1.train()
    model2.train()
    for i, (x,y) in enumerate(ver_train_loader):
        optimizer1.zero_grad()
        optimizer2.zero_grad()

        img1 = x[0].to(device)
        img2 = x[1].to(device)
#         y = y.reshape(-1).to(device) # need to turn to row

        embedding1 = model1(img1)[0]
        embedding2 = model1(img2)[0]
        
        loss = criterion(embedding1, embedding2, y)

        loss.backward()
        optimizer1.step()
        optimizer2.step()

        # progress
        if i%10==0:
            print('Epoch:', epoch, '| Iteration:', i, end='\r')

    
    # Deallocate memory in GPU
    torch.cuda.empty_cache()
    del x
    del y

    # validation
    accuracy = validate(model1, val_loader, val_dataset)
    AUC = verify(model1, ver_val_loader, sub_name)
    val_accuracies.append(accuracy)
    
    if (accuracy > 0.65 or AUC > 0.90):
        save_state(AUC, accuracy, model_number, model1, ver_val_loader_args, device, NUM_EPOCHS, learning_rate, optimizer1, criterion)
    model_number+=1

    scheduler1.step()
    scheduler2.step()

    
    print("Epoch", epoch, "Accuracy:", accuracy, "AUC:", AUC)
    if prev_auc == 0:
        print("\tImprovement:", AUC-prev_auc)
    else:
        print("\tImprovement:", AUC-prev_auc, "| Percent Improvement (times):", 100*AUC/prev_auc, '%')
    # tracking running best AUC
    if running_max[1]<AUC:
        running_max[0]='Model_' + str(RUN_NUMBER) + '_' + str(epoch)
        running_max[1]=AUC
    tf=time()
    print("Time for epoch:", tf-ti)
    print('   Running Max:', *running_max,'\n')

    prev_auc = AUC